# Lab 2

CMSC 457 Spring 2022

Prepared by Yufan Zheng

## Content

* Preliminary
* Q# basics
* Grover's search
* Solving SAT with Grover's search

## Preliminary

* Anaconda: A cross-platform Python distribution
    * Install by the package downloaded from https://www.anaconda.com/download/
* Q# with Jupyter
    * See install guide on https://docs.microsoft.com/en-us/azure/quantum/install-command-line-qdk
    
## Q# Basics

* Grammar resembles that of C#
* Documentation: https://docs.microsoft.com/en-us/azure/quantum/user-guide/
* Do not forget Google-based programming!

### Hello World

`Unit` is a type with only one valid value `()`. Its counterpart in C++ is `void`.

In [ ]:
function HelloQ() : Unit {
    Microsoft.Quantum.Intrinsic.Message("Hello quantum world!"); 
}

* Use `%simulate` command to call an operation or function in Jupyter.
* See https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/ for more magic commands.

In [ ]:
%simulate HelloQ

Use `open` to load libraries, just like what `from ... import *` does in Python or what `#include` does in C++.

In [ ]:
open Microsoft.Quantum.Intrinsic;
    
function HelloQ2() : Unit {
    Message("Hello quantum world (again)!"); 
}

In [ ]:
%simulate HelloQ2

### Variables

* `let` for defining constants
* `mutable` for defining variables
* `set` whenever you change the value of a variable

In [ ]:
open Microsoft.Quantum.Convert; // for IntAsString()

function SetConstant() : Unit {
    let a = 10;
    Message(IntAsString(a));
}

In [ ]:
%simulate SetConstant

In [ ]:
function SetConstant2() : Unit {
    let a = 10;
    set a = a + 1;
    Message(IntAsString(a));
}

In [ ]:
function SetConstant3() : Unit {
    mutable a = 10;
    set a = a + 1;
    Message(IntAsString(a));
}

In [ ]:
%simulate SetConstant3

### Loops and Branching

In [ ]:
function Enumerate() : Unit {
    for i in 0 .. 5 {
        if (i % 3 == 0) {
            Message("A");
        } elif (i % 3 == 1) {
            Message("B");
        } else {
            Message("C");
        }
    }
}

In [ ]:
%simulate Enumerate

### Array

* `b = a w/ i <- x;` means `b = a; b[i] = x;`
* `a w/= i <- x;` means `a[i] = x;`

In [ ]:
function Sum(arr : Int[]) : Int {
    mutable ret = 0;
    for i in 0 .. Length(arr) - 1 {
        set ret = ret + arr[i];
    }
    return ret;
}

function TestSum() : Unit {
    mutable a = [0, size = 4];  // a == [0, 0, 0, 0]
    Message(IntAsString(Sum(a)));
    mutable b = a w/ 2 <- 3;  // b == [0, 0, 3, 0]
    Message(IntAsString(Sum(b)));
    set a w/= 0 <- 4;  // a == [4, 0, 0, 0]
    set a w/= 1 <- 3;  // a == [4, 3, 0, 0]
    set a w/= 2 <- 2;  // a == [4, 3, 2, 0]
    set a w/= 3 <- 1;  // a == [4, 3, 2, 1]
    Message(IntAsString(Sum(a)));
}

In [ ]:
%simulate TestSum

### Qubit and Operation
* Use `use` to get new qubits in the $|0 \rangle$ state
* Qubits _have to_ be restored to $|0 \rangle$, or measured right before release
* Anything related to quantum have to be done in `operation` but not `function`
    * `function` cannot call `operation`
    * Rule of thumb: the output of `function` is deterministic
* Check https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic for basic quantum gates
* `M(q)` for measuring a single qubit `q` with computational basis

In [ ]:
operation DrawACoin() : Result {
    use q = Qubit();
    H(q);
    return M(q); // M(q) == Zero or One
}

operation DrawCoins(n : Int) : Unit {
    mutable res = "";
    for i in 1 .. n {
        if (DrawACoin() == Zero) {
            set res += "H";
        } else {
            set res += "T";
        }
    }
    Message(res);
}

In [ ]:
%simulate DrawCoins n=20

### Operation/Function as Input/Output

In [ ]:
open Microsoft.Quantum.Diagnostics;  // for DumpMachine()

operation ApplyToEach(op : Qubit => Unit, qReg : Qubit[]) : Unit {
    for q in qReg {
        op(q);
    }
}

operation TestApplyToEach() : Unit {
    use qReg = Qubit[3];
    ApplyToEach(H, qReg);
    DumpMachine();
}

In [ ]:
%simulate TestApplyToEach

In [ ]:
function MultiQubitVersionOf(op : Qubit => Unit) : Qubit[] => Unit {
    return ApplyToEach(op, _);
}

operation TestMultiQubitVersionOf() : Unit {
    use qarr = Qubit[2];
    (MultiQubitVersionOf(H))(qarr);
    DumpMachine();
}

In [ ]:
%simulate TestMultiQubitVersionOf

### Controlled and Adjoint Operators

We can let Q# compute the controlled and the adjoint version of _any_ operator.
```
H(q);
Adjoint H(q);
Controlled H(qReg, q);
Adjoint Controlled H(qReg, q);
```
In order to let `Adjoint` and `Controlled` work on user-defined operators, we need to tell Q# explicitly to generate them:
```
operation ApplyT2Each(qReg : Qubit[]) : Unit is Adj + Ctl {
    for q in qReg {
        T(q);
    }
}
```

## Quantum Katas

* https://github.com/microsoft/QuantumKatas
* Official self-paced tutorial of Q#


## Grover's Search

See https://github.com/microsoft/QuantumKatas/blob/56dbd9d806c18693f9bf8f19103e813c9b1fa3e3/GroversAlgorithm/GroversAlgorithm.ipynb.

## Solving SAT with Grover's Search

$$
    O : |x\rangle |y \rangle \mapsto |x\rangle |y \oplus f(x) \rangle
$$
* Our previous construction is $f(x) = \boldsymbol{1}\{x = z\}$ for a designated $z$.
* However, it is possible to set e.g. $f(x) = (x_1 \lor \neg x_2 \lor x_4) \land (x_2 \lor x_3) \land \dots$.
* SAT is unlikely to be solved in time $2^{(1-o(1))n}$ classically, while Grover gives us $\sim \sqrt{2^n} = 2^{0.5n}$.